In [1]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table, ctx
from jupyter_dash import JupyterDash
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as npz
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import numpy as np
from LifeData import LifeData

In [2]:
life_data = LifeData()

In [3]:
def create_life_bar(indicator, state=None):
    
    df = life_data.get_single_indicator(indicator)
    df = df.sort_values(by=["Rate"], ascending=False).reset_index(drop=True)
    colors  =['Blue',]*50
    if state:
        idx = df[df["State"]==state].index[0]
        colors[idx] = "Red"
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df["State"], y=df["Rate"], hovertext=df["Indicator"], marker_color=colors))
    return  fig

    

In [4]:
def life_us_map(indicator):
    df = life_data.get_single_indicator(indicator)
    fig = go.Figure()
    
    fig.add_trace(go.Choropleth(
        locations = df["State"],
        z = df["Rate"],
        locationmode = 'USA-states',
        colorscale='Blues',
        hovertext=df["Indicator"],
        colorbar_x=0,
        marker_line_color='white'
    ))
    fig.update_layout(
        title_text =f"{df.Year.max()} {df.Indicator.unique()[0]} - State Wide",
        geo_scope='usa',
        height=600,
        width=1200,
    )
    fig.update_xaxes(fixedrange=True)
    fig.update_yaxes(fixedrange=True)

    return fig

In [5]:
app = JupyterDash(__name__)

income_life_exp = pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")

app.config.suppress_callback_exceptions = True

app.layout = html.Div(className="row-fluid", children=[
    html.H1(children='USA', style={'textAlign':'center'}),
    html.Div(className="container-fluid", children=[
        html.Div(className = "row", children=[
            html.Div(dcc.Dropdown(['Life', 'Health', 'Substance', 'Wealth', 'Inequality', 'Enviroment', 
                                   'Peace', 'Safety', 'Terrorism', 'Democracy', 'Equality', 'Knowledge', 
                                   'Quality of Life', 'Happiness']),className = "span8 offset2"),
             
        ]),
        html.Div(className = "row", children=[
            html.Div(dcc.RadioItems(["State", "County "], 'State', inline=True), className="span4 offset5")
        ])
        ]),
   
    html.Div(className = "container-fluid", children=[
        html.Div(children=[
            html.Div(dcc.Graph(id='graph-content', figure=life_us_map("Life Exp"), 
                                config={
                                    'displayModeBar': False,
                                    'showTips': False,
                                    }, 
                               className = "span7")),
            html.Div(id = "my-div", className="row", children = [
                html.Div(dash_table.DataTable(id='state-card', page_size=10), 
                         className = "span5"),
                html.Div(dcc.Graph(id='life_histogram', figure=create_life_bar("Life Exp"), config={'displayModeBar': False,'showTips': False,},
                                   className = "span5")),])
            ])
            
    ]),
    html.Div(className = "container-fluid", id = "this-one", children=[html.Div(className = "span5"),]),
    html.Button('Submit', id='your-button', n_clicks=0),
    
])

@callback(
    Output('graph-content', 'figure'),
    Output('state-card', 'data'),
    Output('life_histogram', 'figure'),
    Input('graph-content', 'clickData'),
    Input('state-card', 'active_cell'),
    Input('state-card', 'data'),
    Input('life_histogram', 'clickData'),
    prevent_initial_call=True
)
def data_click(clickData, activeCell, df, histogram_click):
    triggered_id = ctx.triggered_id
    state = clickData["points"][0]["location"]
    if triggered_id == 'graph-content':
        ind = clickData["points"][0]["hovertext"]
    elif triggered_id == 'state-card':
        df = pd.DataFrame.from_dict(df)
        ind = df.iloc[activeCell["row"]]["Indicator"]
    elif triggered_id == 'life_histogram':
        ind = histogram_click["points"][0]["hovertext"]
        state = histogram_click["points"][0]["label"]
        
    print(state)
    table_df = life_data.get_state_card_data(state)
    bar_chart = create_life_bar(ind, state)

    
    return life_us_map(ind), table_df.to_dict('records'), bar_chart


@app.callback(Output('this-one', 'children'),
              [Input('your-button', 'n_clicks')],
              [State('this-one', 'children')])
def add_new_component(n_clicks, div_children):
    # div_children holds the current components of the div

    #add another compenent
    #new_component = dash_table.DataTable(id='table')
    print(n_clicks)
    print(div_children)
    new_component = html.Button('Submit', id='New', n_clicks=0)
    div_children.append(new_component)
    print(div_children)


    return div_children

        
    


if __name__ == '__main__':
    app.run_server(debug=True)
    
    
# https://dash.plotly.com/duplicate-callback-outputs

Dash is running on http://127.0.0.1:8050/



Dash app running on http://127.0.0.1:8050/


In [ ]:
table_df = life_data.get_state_card_data("AL")

In [ ]:
def filter_data_most_recent(self, df):
       idxs = []
       for ind in df.Indicator.unique():
           ind_df = df[df.Indicator==ind]
           ind_df = ind_df[ind_df.Year == ind_df.Year.max()]
           idxs.extend(list(ind_df.index))
       df = df.loc[idxs]
       return df

In [ ]:
import json

In [ ]:
file = open('./cdc_data.json')
meta_files = json.load(file)

In [ ]:
meta_files['CDC Files'][0]

In [ ]:
def clean_data(meta_file):
    
    df = pd.read_csv(meta_file['File Location'])
    df["Indicator"] = meta_file['Indicator Name']
    df = df.rename(columns={"YEAR":"Year","STATE":"State", meta_file['Rate Name']:"Rate"})
    if meta_file['Missing Data']:
        df["Rate"] = df["Rate"].replace(meta_file['Missing Data'], 0.0)
    df["Rate"] = df["Rate"].astype(float)
    df = df[["State", "Year", "Indicator", "Rate"]]
    df = get_rank_year(df)
    return df



In [ ]:
meta_files['CDC Files'][0]['Rate Name']

In [ ]:
clean_data(meta_files['CDC Files'][1])